In [1]:
%run ../../utils/commonImports.py
%run ../../utils/tradingImports.py
%matplotlib inline

from base import supres, BaseStrategy

# Load Data

In [2]:
# dropbox_dir = 'D:\\Dropbox\\My work\\krypl-project'
dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'

def read_data(pair):
    db = os.path.join(dropbox_dir,'sqlite', 'ploniex-chart-data', f'{pair}.db')
    data = load_trading_data(db, 'chart_data', from_date='2015-01-01', period='30min')
    data['ohlc4'] = (data['open'] + data['close'] + data['high'] + data['low']) / 4
    return data

In [5]:
supports_for_data_dict = load_model('supports-for-data.pkl')

In [4]:
features = ['support']

In [3]:
train_pairs = pd.read_csv('pairs_train.tsv', header=None)[0].tolist()
test_pairs = pd.read_csv('pairs_test.tsv', header=None)[0].tolist()
all_pairs = pd.read_csv('pairs.tsv', header=None)[0].tolist()
data_dict = {pair: read_data(pair) for pair in all_pairs}

# Parallelize

In [6]:
from multiprocessing import Pool
import time
from tqdm import *

def run_parallel(f, args, n_process=10):
    results = []
    with Pool(processes=n_process) as p:
        all_args = list(args)
        with tqdm_notebook(enumerate(p.imap_unordered(f, all_args)), leave=False, total=len(all_args)) as pbar:
            for i, x in pbar:
                results.append(x)
                pbar.update()
                
    return results

In [7]:
def dict_from_list(l):
    d = {}
    for row in l:
        d_inner = d
        for k in row[:-2]:
            if k not in d_inner.keys():
                d_inner[k] = {}
            d_inner = d_inner[k]
        d_inner[row[-2]] = row[-1]
    return d

In [8]:
window_sizes = [4, 6, 8, 10, 12, 24, 30, 48]

# Add Test Supports

In [74]:
def get_supports(data, window_size):
    supports, _ = supres(data['ohlc4'].values, window_size)
    supports = pd.DataFrame(supports, columns=['index', 'support'])
    support_vals = supports['support'].values
    size = support_vals.shape[0]
    _return = (support_vals[1:] / support_vals[:size-1]) - 1
    _return = np.insert(_return, 0, 0)
    supports['return1'] = _return
    return supports

In [ ]:
from itertools import product

def supports_f(args):
    pair, window_size = args
    return pair, window_size, get_supports(data_dict[pair], window_size)

args = product(test_pairs, window_sizes)
results = run_parallel(supports_f, args)
supports_dict_test = dict_from_list(results)
save_model(supports_dict_test, 'supports_test.pkl')

# Data With Supports

In [9]:
supports_dict_test = load_model('supports_test.pkl')

In [10]:
from itertools import product

def find_support(i, supports):
    selected = supports[supports['index'] <= i]['support']
    return None if len(selected) == 0 else selected.iloc[-1]


def supports_f(args):
    pair, window_size = args
    supports = supports_dict_test[pair][window_size]
    data = data_dict[pair]
    s = [find_support(i, supports) for i in range(len(data))]
    return pair, window_size, s

args = product(test_pairs, window_sizes)
results = run_parallel(supports_f, args)
supports_for_data_dict_test = dict_from_list(results)
save_model(supports_for_data_dict_test, 'supports-for-data-test.pkl')

In [12]:
supports_for_data_dict_test = load_model('supports-for-data-test.pkl')
supports_for_data_dict_train = load_model('supports-for-data.pkl')
supports_for_data_dict = {**supports_for_data_dict_train, **supports_for_data_dict_test}

In [ ]:
save_model(supports_for_data_dict, 'supports-for-data-all.pkl')